In [1]:
from keras.models import Model
from keras.layers import Lambda,Activation,Dense,Conv2D,Input,BatchNormalization,MaxPooling2D,Flatten
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
input_tensor_1 = Input([32,32,3])
input_tensor_2 = Input([4,])
input_target = Input([2,])

In [3]:
def cus_loss1(y_true,y_pred):
    return K.mean(K.abs(y_true - y_pred))

def cus_loss2(y_true,y_pred):
    return K.mean(K.abs(y_true - y_pred))

In [4]:
#网络结构
# 第一组
x = BatchNormalization(axis=1)(input_tensor_1)

x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x=Flatten()(x)
x=Dense(units=16)(x)
out2 = Dense(units=2)(x)


# 第二组
y = Dense(units=32)(input_tensor_2)
out1 = Dense(units=2)(y)

# 第三组
z = Dense(units=8)(input_target)
out3 = Dense(units=2)(z)


#自定义loss
loss1 = Lambda(lambda x:cus_loss1(*x),name='loss1')([out2,out1]) # out2-out1 = loss
loss2 = Lambda(lambda x:cus_loss2(*x),name='loss2')([out3,out2])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#model = Model(inputs=[input_tensor_1,input_tensor_2,input_target],outputs=[out1,out2,out3])

model = Model(inputs=[input_tensor_1,input_tensor_2,input_target],outputs=[out1,out2,out3,loss1,loss2])

In [6]:
#获取loss，创建loss值
loss_layer1 = model.get_layer('loss1').output
loss_layer2 = model.get_layer('loss2').output

# 向模型中添加loss
model.add_loss(loss_layer1)
model.add_loss(loss_layer2)

In [7]:
#loss + None 表示不需要梯度返回，有多少个out就有多少个none
model.compile(optimizer='sgd',loss=[None,None,None,None,None])

In [20]:
def data_gen(number):
    for i in range(number):
        yield [np.random.normal(1,1,size=(1,32,32,3)),
               np.random.normal(1,1,size=(1,4)),
              np.random.normal(1,1,size=(1,2))],[]
                               
                               
dataset = data_gen(1000)
    

In [32]:
#dataset.__next__()

[array([[[ 0.96748074,  1.62591756,  1.85918866],
         [-0.23078913,  0.9336379 , -0.1807498 ],
         [ 0.90899228,  1.62695903,  0.82880737],
         ...,
         [ 0.3311553 ,  1.25494759,  0.92462397],
         [ 1.85218305,  1.03963725, -0.99561408],
         [ 0.95749449,  0.29273096,  1.45851842]],
 
        [[ 0.26756502,  0.82215038,  1.46171536],
         [ 1.0006223 ,  0.96684151, -0.13613011],
         [ 0.04917444, -0.94709108,  2.20017396],
         ...,
         [ 1.37476895,  0.13021441,  1.12182977],
         [ 1.56017374,  1.36890538,  1.39018349],
         [ 0.15802545,  1.44646388,  0.89216951]],
 
        [[ 0.6369584 , -0.3897463 ,  2.01878003],
         [ 2.35115683,  0.0454149 ,  0.5687246 ],
         [ 1.54437376, -0.30343867,  1.72664443],
         ...,
         [ 0.03540871,  2.9781393 ,  1.37218957],
         [ 0.85727742,  0.96613206,  0.47021767],
         [ 0.51650773,  1.64768066,  3.11413702]],
 
        ...,
 
        [[ 1.94018663,  0.47392688

In [21]:
#train = model.fit_generator(dat)
train = model.fit_generator(dataset,epochs=10,steps_per_epoch=20)

Epoch 1/10
20/20 [==============================] - 1s 36ms/step - loss: 3.1196
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 1.1308
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6121
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.3183
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.2387
Epoch 6/10
20/20 [==============================] - ETA: 0s - loss: 0.166 - 0s 3ms/step - loss: 0.1610
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1865
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1575
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1470
Epoch 10/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1279


In [9]:
from keras.utils.vis_utils import plot_model

plot_model(model=model,to_file='model_plain_loss.png',show_shapes= True)